In [1]:
%cd D:/

D:\


In [6]:
pip install tensorflow


  Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl (300.9 MB)
     ---------------------------------------- 5.5/5.5 MB 132.0 kB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl (413 kB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-win_amd64.whl (938 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl (1.5 MB)
     -------------------------------------- 24.4/24.4 MB 294.1 kB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ------------------------------------ 442.0/442.0 kB 310.6 kB/s eta 0:00:00
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
     ---------------------------------------- 3.8/3.8 MB 166.5 kB/s eta 0:00:00
  Using cached keras-2.15.0-p

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras import layers,models

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
IS=256
channel=3
BS=8

In [ ]:
ds_train=tf.keras.preprocessing.image_dataset_from_directory(
    "D:Jamaludeen2/Retinal Fundus Images/train",
    batch_size=BS)
ds_test=  tf.keras.preprocessing.image_dataset_from_directory(
    "D:Jamaludeen2/Retinal Fundus Images/test",
    batch_size=BS)
ds_val=tf.keras.preprocessing.image_dataset_from_directory(
    "D:Jamaludeen2/Retinal Fundus Images/val",
    batch_size=BS)

In [ ]:
classes=ds_train.class_names

In [ ]:
classes

# preprocessing 

In [ ]:
ds_train=ds_train.shuffle(1000)
ds_train=ds_train.take(1000)
ds_train=ds_train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
ds_test=ds_test.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
ds_val=ds_val.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale=tf.keras.Sequential(
    [layers.experimental.preprocessing.Resizing(IS,IS),
    layers.experimental.preprocessing.Rescaling(1.0/255)
    ])

DATA AUGMENTATION

In [ ]:
data_augmentation=tf.keras.Sequential(
    [layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
     layers.experimental.preprocessing.RandomRotation(0.2),
     layers.experimental.preprocessing.RandomContrast((0.1,0.3))

])

# Model pipelining

In [23]:
input_shape=(BS,IS,IS,channel)

In [ ]:
model=models.Sequential([
            data_augmentation,
            layers.Resizing(256, 256),
            layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
            layers.MaxPooling2D((2,2)),
            layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
            layers.MaxPooling2D((2,2)),
            layers.Conv2D(64,kernel_size=(3,3),activation='LeakyReLU'),
            layers.MaxPooling2D((2,2)),
            layers.Conv2D(64,(3,3),activation='LeakyReLU'),
            layers.MaxPooling2D((2,2)),
            layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
            layers.MaxPooling2D((2,2)),
            layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
            layers.MaxPooling2D((2,2)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(11, activation='softmax'),])

In [ ]:
model.build(input_shape=input_shape)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
history=model.fit(ds_train,batch_size=BS,epochs=15,verbose=1,validation_data=ds_val)
history.params
history.history.keys()
        
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

In [ ]:
def predict(model,img):
    img_array=tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array=tf.expand_dims(img_array,0)
    
    predictions=model.predict(img_array)
    
    predicted_class=classes[np.argmax(predictions[0])]
    confidence=round(100*(np.max(predictions[0])),2)
    
    return predicted_class,confidence

In [ ]:
plt.figure(figsize=(15,15))
for images,labels in ds_test.take(3):
    for i in range(10):
        ax=plt.subplot(2,4,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class,confidence=predict(model,images[i].numpy())
        actual_class=classes[labels[i]]
        
        plt.title(f"Actual:{actual_class},\n Predicted:{predicted_class}.\n confidence:{confidence}%")
        plt.axis("off")

In [ ]:
model.save("Jamaludeen2/retinalmodel")

In [7]:
#Resnet (Residual sequetial network)

In [12]:
from tensorflow.keras.applications import ResNet50

In [32]:
import keras.backend as K
K.set_floatx('float64')

In [14]:
IS=256
channel=3
BS=8

In [25]:
input_shape=(IS,IS,channel)

In [ ]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
for layer in resnet_model.layers:
    layer.trainable = False

In [34]:
model1=models.sequential([
    resenet_model,
    model,
])

AttributeError: module 'keras.api._v2.keras.models' has no attribute 'sequential'